In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [ ]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

trip = pd.read_csv('서울_final_data.csv')
trip = trip.loc[:, ['keyword', 'name']]

# trip['name'] = trip['name'].str.replace("[^\w]", "")
trip['keyword'] = trip['keyword'].str.replace("[^\w]", " ")

# # 문자열 -> 딕셔너리 형태로 변환
# # trip['name'] = trip['name'].apply(literal_eval)
# trip['keyword'] = trip['keyword'].apply(literal_eval)

# # i = 0
# # for row in trip['name']:
# #     name = []
# #     for ele in row:
# #         name.append(ele['name'])
# #     trip['name'][i] = name
# #     i = i + 1

# # 딕셔너리 형태를 깔끔하게 장르에 해당하는 부분만 뽑아서 문자열화
# # [{}, {}, {}, {}] -> [장르, 장르, 장르, 장르]
# # trip['name'] = trip['name'].apply(lambda x : [y['name'] for y in x])
# trip['keyword'] = trip['keyword'].apply(lambda x : [y['name'] for y in x])

# # [장르, 장르, 장르, 장르] -> 장르 장르 장르 장르
# # trip['name'] = trip['name'].apply(lambda x : ' '.join(x))
# trip['keyword'] = trip['keyword'].apply(lambda x : ' '.join(x))

print(trip)

                                                keyword          name
0        도서관    도심여행    문화시설    문화행사    사계절      아이와...      (주) 교보문고
1                       관광지    역사속으로    역사이야기    자연여행    150년 수령 느티나무
2       문화공간    문화시설    문화예술    복합문화공간    아이와함께    열...         1m아트홀
3              관광지    사계절      아이와함께    역사공부    역사관광지      3.1독립선언기념탑
4                                관광지    역사공부    역사관광지      3.1독립운동기념탑
...                                                 ...           ...
1342    관광지    산책길    산책하기    산책하기좋은곳    역사공부    역사관...         훈련원공원
1343    가족여행    관광지    교과서속여행    당일치기여행    근교여행    아...          흥인지문
1344    관광지    불교    불교사찰    사찰    사찰여행    역사    자연속...           흥천사
1345    공연장    남녀노소    문화공간    문화시설    복합문화공간    연인과...       흰물결아트센터
1346                                  쇼핑    외국인친구에게추천           힐튼양복점

[1347 rows x 2 columns]


# TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# ngram_range=(1, 2) 는 단어를 1개 혹은 2개 연속으로 보겠다
tfidf_vec = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vec.fit_transform(trip['keyword'])
print(tfidf_vec.vocabulary_.items())
# 14061은 여행지의 개수, 276은 단어의 개수 -> 하나의 영화를 276개 열을 가진 벡터로 표현
print(tfidf_matrix.shape)

# action adventure fantasy
# adventure fantasy
# adventure
# fantasy

dict_items([('도서관', 896), ('도심여행', 931), ('문화시설', 1251), ('문화행사', 1401), ('사계절', 1654), ('아이와함께', 2108), ('도서관 도심여행', 898), ('도심여행 문화시설', 947), ('문화시설 문화행사', 1273), ('문화행사 사계절', 1405), ('사계절 아이와함께', 1672), ('관광지', 351), ('역사속으로', 2306), ('역사이야기', 2312), ('자연여행', 2500), ('관광지 역사속으로', 473), ('역사속으로 역사이야기', 2308), ('역사이야기 자연여행', 2313), ('문화공간', 1214), ('문화예술', 1323), ('복합문화공간', 1537), ('열린문화공간', 2350), ('문화공간 문화시설', 1218), ('문화시설 문화예술', 1258), ('문화예술 복합문화공간', 1330), ('복합문화공간 아이와함께', 1561), ('아이와함께 열린문화공간', 2136), ('역사공부', 2261), ('역사관광지', 2272), ('관광지 사계절', 446), ('아이와함께 역사공부', 2129), ('역사공부 역사관광지', 2263), ('관광지 역사공부', 472), ('가족과함께', 57), ('데이트코스', 870), ('가볼만한곳', 38), ('가족과함께 관광지', 77), ('관광지 데이트코스', 402), ('데이트코스 복합문화공간', 884), ('복합문화공간 가볼만한곳', 1538), ('경치좋은곳', 227), ('미술관', 1437), ('미술작품', 1456), ('경치좋은곳 문화공간', 230), ('문화예술 미술관', 1328), ('미술관 미술작품', 1443), ('남녀노소', 727), ('부모님과함께', 1598), ('스파', 2005), ('연인과함께', 2324), ('이색체험', 2461), ('관광지 남녀노소', 386), ('남녀노소 부모님과함께', 761), ('부모님과함께 

In [ ]:
# 유사도 행렬 (4803, 4803)
# 1, 1 (1번째 영화와 1번재 영화의 유사도)
# 1, 1 / 1, 2 / .... / 1, 4803 -> 1번째 영화와 1~4803번재 영화의 유사도
# 2, 1 / 2, 2 / .... / 2, 4803 -> 2번째 영화와 1~4803번째 영화의 유사도
# ....
# 4803, 1 / 4803, 2 / .... / 4803, 4803 -> 4803번째 영화와 1~4803번째 영화의 유사도
from sklearn.metrics.pairwise import cosine_similarity

# 4803개의 영화랑 4803개의 영화끼리 유사도를 구하겠다!
# 자신과의 유사도는 1
genres_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(genres_similarity)
# 유사도 값이 높은 영화의 제목
# 유사도 값이 높은 순으로 인덱스 값을 뽑아낸다
similar_index = np.argsort(-genres_similarity)
print(similar_index)

[[1.         0.         0.05837155 ... 0.         0.01753124 0.        ]
 [0.         1.         0.         ... 0.01159073 0.         0.        ]
 [0.05837155 0.         1.         ... 0.         0.2312937  0.        ]
 ...
 [0.         0.01159073 0.         ... 1.         0.         0.        ]
 [0.01753124 0.         0.2312937  ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
[[   0 1324  836 ...  569  552 1346]
 [   1  792  775 ...  535  498 1346]
 [   2 1334 1241 ...  621  604 1346]
 ...
 [1344 1329  813 ...  545  502 1346]
 [1345  679  675 ...  743  730 1346]
 [1346 1307 1299 ...  504  511  673]]


# Count (빈도수)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(ngram_range=(1, 2))
count_matrix = count_vec.fit_transform(trip['keyword'])
print(count_matrix)

  (0, 896)	1
  (0, 931)	1
  (0, 1251)	1
  (0, 1401)	1
  (0, 1654)	1
  (0, 2108)	1
  (0, 898)	1
  (0, 947)	1
  (0, 1273)	1
  (0, 1405)	1
  (0, 1672)	1
  (1, 351)	1
  (1, 2306)	1
  (1, 2312)	1
  (1, 2500)	1
  (1, 473)	1
  (1, 2308)	1
  (1, 2313)	1
  (2, 1251)	1
  (2, 2108)	1
  (2, 1214)	1
  (2, 1323)	1
  (2, 1537)	1
  (2, 2350)	1
  (2, 1218)	1
  :	:
  (1344, 2497)	1
  (1344, 443)	1
  (1344, 1626)	1
  (1344, 1710)	1
  (1344, 1718)	1
  (1344, 2255)	1
  (1344, 2501)	1
  (1344, 2498)	1
  (1344, 1616)	1
  (1345, 1251)	1
  (1345, 1214)	1
  (1345, 1537)	1
  (1345, 2350)	1
  (1345, 1218)	1
  (1345, 727)	1
  (1345, 2324)	1
  (1345, 1286)	1
  (1345, 753)	1
  (1345, 1564)	1
  (1345, 282)	1
  (1345, 2327)	1
  (1345, 289)	1
  (1346, 1882)	1
  (1346, 2414)	1
  (1346, 1911)	1


In [ ]:
# 유사도 행렬 (4803, 4803)
# 1, 1 (1번째 영화와 1번째 영화의 유사도)
# 1, 1 / 1, 2 / .... / 1, 4803
# 2, 1 / 2, 2 / .... / 2, 4803
# .....
# 4803, 1 / 4803, 2 / .... / 4803, 4803
from sklearn.metrics.pairwise import cosine_similarity

# 4803개의 영화랑 4803개의 영화끼리 유사도를 구하겠다!
genres_similarity = cosine_similarity(count_matrix, count_matrix)
print(genres_similarity)
# 유사도가 높은 영화를 알고 싶다!
# 유사도 값이 높은 것의 인덱스를 내림차순으로 출력/리턴
similar_index = np.argsort(-genres_similarity)
print(similar_index)

[[1.         0.         0.18181818 ... 0.         0.0836242  0.        ]
 [0.         1.         0.         ... 0.09759001 0.         0.        ]
 [0.18181818 0.         1.         ... 0.         0.41812101 0.        ]
 ...
 [0.         0.09759001 0.         ... 1.         0.         0.        ]
 [0.0836242  0.         0.41812101 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
[[   0  836 1324 ...  569  552 1346]
 [   1  792  179 ...  535  498 1346]
 [   2 1334  304 ...  634  636 1346]
 ...
 [1329 1344  813 ...  545  502 1346]
 [1345  679  675 ...  297  294 1346]
 [1346  389 1299 ...  504  511  673]]


In [ ]:
keyword = input().split()
# keyword = keyword.split()

length = len(keyword)-3
if length <= 0:
    length = 1

print(length)
name = []

for i, t in enumerate(trip['keyword']):
    for k in keyword:
        if k in t:
            name.append(trip['name'][i])

name.sort()
name_count= {}
for n in name:
    try: name_count[n]+= 1
    except: name_count[n]=1

value = pd.DataFrame(list(name_count.items()),
                   columns=['name', 'count'])
travel = value[value['count'] >= length]
# travel

산책
1


In [ ]:
# 사용자가 입력한 영화의 인덱스 값을 찾아내고
# similar_index 에 기록된 유사한 영화 인덱스를 찾아내고
# 유사한 영화 인덱스를 토대로 영화 이름을 찾아내면 된다!
# input_trip = input()
for n in travel['name']:
    trip_index = trip[trip['name']==n].index.values
    print(trip_index)
    similar_trip = similar_index[trip_index, :2]
    print(similar_trip)
    # 인덱스로 사용하기 위해서는 1차원으로 변형해줘야하기 때문
    similar_trip_index = similar_trip.reshape(-1)
    print(similar_trip_index)
    print(trip.iloc[similar_trip_index])

[40]
[[40 41]]
[40 41]
                                             keyword                      name
40             레포츠    산책로    산책코스    산책하기좋은곳    산책    [북악하늘길 1산책로] 북악팔각정~말바위쉼터
41    레포츠     산책로    산책코스    산책하기좋은곳    산책    힐링산책     [북악하늘길 2산책로] 하늘교~성북천발원지
[41]
[[41 40]]
[41 40]
                                             keyword                      name
41    레포츠     산책로    산책코스    산책하기좋은곳    산책    힐링산책     [북악하늘길 2산책로] 하늘교~성북천발원지
40             레포츠    산책로    산책코스    산책하기좋은곳    산책    [북악하늘길 1산책로] 북악팔각정~말바위쉼터
[42]
[[42 54]]
[42 54]
                                              keyword                   name
42    레포츠    부모님과함께    산책로    산책코스    산책하기좋은곳    자...  [북악하늘길 3산책로] 북카페~숲속다리
54    등산    등산객    레포츠    산림욕    산책로    산책코스    산책...        [북한산 둘레길] 9 마실길
[43]
[[44 43]]
[44 43]
                            keyword                      name
44     레포츠    북악하늘길    트레킹    산책로     [북악하늘길 형제봉오름길] 하늘마루~형제봉
43    레포츠    북악하늘길     트레킹    산책로    [북악하늘길 스카이웨이] 하늘한마당~하늘마루
[44]
[[44 43]]
[4